# Multi i-TED characterization
## Fifth: Hardware updates

- In the [previous notebook](./PyROOT_resolution_Cs137v4.html) hardware updates were proposed in order to:
    - Fix light distributions effects
    - Improve the resolution of the worst crystals
- The hsitograms are normalized and the background is subtracted to improve the results
- Multiple runs were performed in series in order
- Studies of alpha activity are included

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

uproot              4.3.5
Welcome to JupyROOT 6.28/02
sklearn             1.2.2
torch               2.0.0
scipy               1.10.1
numpy               1.23.5
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.62.3


In [2]:
%jsroot

In [3]:
class spectrum:
    
    def __init__(self, TH1D_, iTED_, Crystal_, Configuration_, Window_, Calibration_):  

        self.__TH1D = TH1D_
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Window = Window_
        self.__Calibration = Calibration_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def __ch__(self, en):
        p = numpy.poly1d(self.__Calibration[::-1])
        temp  = set(i for i in (p - en).roots if i > 0)
        return list(temp).pop()
    
    def TH1D(self):
        return self.__TH1D
    
    def Calibration(self):
        return self.__Calibration
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Window(self):
        return self.__Window
    
    def Rate(self, Time_):
        return self.__TH1D.Integral()/Time_
    
    def Alpha(self, Time_):
        return self.__TH1D.Integral(
            self.__TH1D.FindBin(self.__ch__(1600)),
            self.__TH1D.FindBin(self.__ch__(2800))
        )/Time_
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {},Window: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )
    
    def __str__(self):
        return "{}.{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )

In [4]:
iTEDA_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDA.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDB_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDB.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDC_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDC.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDD_cal = pandas.read_csv(
    "../../data/2023-03-02/Energy_Calibrations_02_03_2023/Energy_Calibrations_iTEDD.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTED_cal = pandas.concat([iTEDA_cal.T, iTEDB_cal.T, iTEDC_cal.T, iTEDD_cal.T], axis=1, keys=['A', 'B', 'C', 'D'])

iTED_cal

A                                                          B  \
           0           1           2          3           4           0   
P0  97.91440  138.744000  164.131000  96.575700  184.627000  123.545000   
P1   1.96942    1.813160    2.304510   1.764130    2.327000    2.228220   
P2   0.00042    0.000665    0.001125   0.000595    0.001193    0.000559   

                                                             C              \
             1           2           3           4           0           1   
P0  181.753000  164.939000  162.539000  156.664000  115.328000  147.053000   
P1    2.581200    2.116580    2.010630    2.193700    2.179250    2.005310   
P2    0.001048    0.001154    0.000915    0.000946    0.000469    0.000859   

                                                D                          \
             2           3           4          0           1           2   
P0  129.204000  144.779000  132.346000  91.325100  138.429000  121.824000   
P1    2.079170    2.152760    2.311890   2.016440    2.065850    2.071090   
P2    0.000717    0.000836    0.000898   0.000479    0.000809    0.000667   

                            
             3           4  
P0  138.067000  122.738000  
P1    2.132460    1.964320  
P2    0.000949    0.000705

In [5]:
config = ["888"]
CW = [100,150,200,250]
run = [1,2,3]

ited = ["A","B","C","D"]
crystal = [0,1,2,3,4]

run = [1,2,3]

midx = pandas.MultiIndex.from_product(
    [run, config, CW],
    names=['Run', 'Configuration', 'Window']
)

mcol = pandas.MultiIndex.from_product(
    [ited, crystal],
    names=['iTED', 'Crystal']
) 

spectra = pandas.DataFrame(index = midx, columns = mcol)

# Access using spectra[iTED,Crystal][Configuration,Window]

In [6]:
iTEDA = {
    "888-100-1": ROOT.TFile.Open("/run/media/bgameiro/92A6-F521/Multi_iTED_HWUpdate/Cs137_iTEDA_Crystal4ch_SiPM4ch_D.2023_04_25_T.16_47_39_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150-1": ROOT.TFile.Open("/run/media/bgameiro/92A6-F521/Multi_iTED_HWUpdate/Cs137_iTEDA_Crystal4ch_SiPM4ch_D.2023_04_25_T.16_47_39_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200-1": ROOT.TFile.Open("/run/media/bgameiro/92A6-F521/Multi_iTED_HWUpdate/Cs137_iTEDA_Crystal4ch_SiPM4ch_D.2023_04_25_T.16_47_39_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250-1": ROOT.TFile.Open("/run/media/bgameiro/92A6-F521/Multi_iTED_HWUpdate/Cs137_iTEDA_Crystal4ch_SiPM4ch_D.2023_04_25_T.16_47_39_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
    "888-100-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDA_A2_D.2023_04_20_T.16_19_07_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDA_A2_D.2023_04_20_T.16_19_07_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDA_A2_D.2023_04_20_T.16_19_07_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDA_A2_D.2023_04_20_T.16_19_07_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
}

iTEDB = {
    "888-100-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDB-HWUpdate_D.2023_04_11_T.12_37_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW100.root","READ"),
    "888-150-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDB-HWUpdate_D.2023_04_11_T.12_37_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW150.root","READ"),
    "888-200-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDB-HWUpdate_D.2023_04_11_T.12_37_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW200.root","READ"),
    "888-250-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDB-HWUpdate_D.2023_04_11_T.12_37_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW250.root","READ"),
    "888-100-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDB_A2_D.2023_04_20_T.16_24_57_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDB_A2_D.2023_04_20_T.16_24_57_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDB_A2_D.2023_04_20_T.16_24_57_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDB_A2_D.2023_04_20_T.16_24_57_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
}

iTEDC = {
    "888-100-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDC-HWUpdate_D.2023_04_11_T.13_27_32_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW100.root","READ"),
    "888-150-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDC-HWUpdate_D.2023_04_11_T.13_27_32_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW150.root","READ"),
    "888-200-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDC-HWUpdate_D.2023_04_11_T.13_27_32_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW200.root","READ"),
    "888-250-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDC-HWUpdate_D.2023_04_11_T.13_27_32_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW250.root","READ"),
    "888-100-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDC_A1_D.2023_04_20_T.16_32_15_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDC_A1_D.2023_04_20_T.16_32_15_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDC_A1_D.2023_04_20_T.16_32_15_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDC_A1_D.2023_04_20_T.16_32_15_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
}

iTEDD = {
    "888-100-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDD-HWUpdate_D.2023_04_11_T.14_10_17_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW100.root","READ"),
    "888-150-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDD-HWUpdate_D.2023_04_11_T.14_10_17_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW150.root","READ"),
    "888-200-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDD-HWUpdate_D.2023_04_11_T.14_10_17_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW200.root","READ"),
    "888-250-1": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137-iTEDD-HWUpdate_D.2023_04_11_T.14_10_17_C.itedABCD_lab_2023.02.22_4.0v_888_300s_serie1_3_CW250.root","READ"),
    "888-100-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDD_A1_D.2023_04_20_T.16_40_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDD_A1_D.2023_04_20_T.16_40_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDD_A1_D.2023_04_20_T.16_40_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250-2": ROOT.TFile.Open("../../data/Multi_iTED_HWUpdate/Cs137_iTEDD_A1_D.2023_04_20_T.16_40_05_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ"),
}

In [7]:
for iTED in[0,1,2,3]:
    for Crystal in [0,1,2,3,4]:
        for Configuration in ["888"]:
            for CW in [100,150,200,250]:
                for run in [1,2]:
            
                    spectra[
                        ["A","B","C","D"][iTED], Crystal
                    ][
                        run, Configuration, CW
                    ] = spectrum(
                        [iTEDA,iTEDB,iTEDC,iTEDD][iTED][f"{Configuration}-{CW}-{run}"].Get(
                            "{}_{}_amplitude_spectra;1".format(
                                "SCATTERER" if Crystal==0 else "ABSORBER",
                                ["A","B","C","D"][iTED] if Crystal==0 else "{}_{}".format(["A","B","C","D"][iTED],Crystal)
                            )
                        ), 
                        ["A","B","C","D"][iTED], 
                        Crystal, 
                        Configuration, 
                        CW, 
                        iTED_cal[:][["A","B","C","D"][iTED],Crystal]
                    )

In [8]:
spectra.dropna(inplace=True)
spectra.applymap(str)

iTED                                A                                         \
Crystal                             0            1            2            3   
Run Configuration Window                                                       
1   888           100     A.0.888.100  A.1.888.100  A.2.888.100  A.3.888.100   
                  150     A.0.888.150  A.1.888.150  A.2.888.150  A.3.888.150   
                  200     A.0.888.200  A.1.888.200  A.2.888.200  A.3.888.200   
                  250     A.0.888.250  A.1.888.250  A.2.888.250  A.3.888.250   
2   888           100     A.0.888.100  A.1.888.100  A.2.888.100  A.3.888.100   
                  150     A.0.888.150  A.1.888.150  A.2.888.150  A.3.888.150   
                  200     A.0.888.200  A.1.888.200  A.2.888.200  A.3.888.200   
                  250     A.0.888.250  A.1.888.250  A.2.888.250  A.3.888.250   

iTED                                             B                            \
Crystal                             4            0            1            2   
Run Configuration Window                                                       
1   888           100     A.4.888.100  B.0.888.100  B.1.888.100  B.2.888.100   
                  150     A.4.888.150  B.0.888.150  B.1.888.150  B.2.888.150   
                  200     A.4.888.200  B.0.888.200  B.1.888.200  B.2.888.200   
                  250     A.4.888.250  B.0.888.250  B.1.888.250  B.2.888.250   
2   888           100     A.4.888.100  B.0.888.100  B.1.888.100  B.2.888.100   
                  150     A.4.888.150  B.0.888.150  B.1.888.150  B.2.888.150   
                  200     A.4.888.200  B.0.888.200  B.1.888.200  B.2.888.200   
                  250     A.4.888.250  B.0.888.250  B.1.888.250  B.2.888.250   

iTED                                                          C               \
Crystal                             3            4            0            1   
Run Configuration Window                                                       
1   888           100     B.3.888.100  B.4.888.100  C.0.888.100  C.1.888.100   
                  150     B.3.888.150  B.4.888.150  C.0.888.150  C.1.888.150   
                  200     B.3.888.200  B.4.888.200  C.0.888.200  C.1.888.200   
                  250     B.3.888.250  B.4.888.250  C.0.888.250  C.1.888.250   
2   888           100     B.3.888.100  B.4.888.100  C.0.888.100  C.1.888.100   
                  150     B.3.888.150  B.4.888.150  C.0.888.150  C.1.888.150   
                  200     B.3.888.200  B.4.888.200  C.0.888.200  C.1.888.200   
                  250     B.3.888.250  B.4.888.250  C.0.888.250  C.1.888.250   

iTED                                                                       D  \
Crystal                             2            3            4            0   
Run Configuration Window                                                       
1   888           100     C.2.888.100  C.3.888.100  C.4.888.100  D.0.888.100   
                  150     C.2.888.150  C.3.888.150  C.4.888.150  D.0.888.150   
                  200     C.2.888.200  C.3.888.200  C.4.888.200  D.0.888.200   
                  250     C.2.888.250  C.3.888.250  C.4.888.250  D.0.888.250   
2   888           100     C.2.888.100  C.3.888.100  C.4.888.100  D.0.888.100   
                  150     C.2.888.150  C.3.888.150  C.4.888.150  D.0.888.150   
                  200     C.2.888.200  C.3.888.200  C.4.888.200  D.0.888.200   
                  250     C.2.888.250  C.3.888.250  C.4.888.250  D.0.888.250   

iTED                                                                          
Crystal                             1            2            3            4  
Run Configuration Window                                                      
1   888           100     D.1.888.100  D.2.888.100  D.3.888.100  D.4.888.100  
                  150     D.1.888.150  D.2.888.150  D.3.888.150  D.4.888.150  
                  200     D.1.888.200  D.2.888.200  D.3.888.200  D.4.888.200  
  

In [9]:
def get_resolution(cell):
    
    TH1D = cell.TH1D()
    
    TH1D.GetXaxis().SetRange(TH1D.FindBin(100),TH1D.FindBin(400))
    
    MaxBin   = TH1D.FindBin(TH1D.GetMaximumBin())
    
    ADC_Low  = MaxBin-60
    ADC_High = MaxBin+40

    gaussFit = ROOT.TF1("gaussFit", "pol1(0)+gaus(2)", ADC_Low, ADC_High)
    gaussFit.SetParameters(1000,-1,TH1D.GetMaximum(),MaxBin,7)
    TH1D.Fit(gaussFit,"QR")
    
    sigma = abs(gaussFit.GetParameter(4))
    centroid_ch = gaussFit.GetParameter(3)
    
    x1 = cell(centroid_ch+sigma*numpy.sqrt(2*numpy.log(2)))
    x2 = cell(centroid_ch-sigma*numpy.sqrt(2*numpy.log(2)))
    centroid = cell(centroid_ch)
        
    fwhm = x1-x2
            
    return sigma*numpy.sqrt(2*numpy.log(2))*2/centroid_ch*100, (fwhm/centroid)*100, centroid

In [10]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)
    
    l4 = uproot.open(f"../../data/nTOF_March2022/888/CW100ns/Resolutions_Cs137_CenterScatter_iTED{cell.iTED()}_8.8.8_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    if l4*0.9 < l2 < l4*1.1:
        latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
    else:
        latex.DrawText(0.7, 0.75, "->R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
        
    latex.DrawText(0.7, 0.65, "R_E(old): {:.2f}%".format(l4))
    latex.DrawText(0.7, 0.6, "Change: {:.2f}%".format(l2-l4))
            
    return canvas

## Analysis

### Energy resolution @662keV

#### Using channel values

In [11]:
spectra.applymap(lambda x: get_resolution(x)[0]).T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


#### Using the current calibration (March 2023, 888, 100ns)

In [12]:
resolution = spectra.applymap(lambda x: get_resolution(x)[1])

Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 


In [13]:
resolution.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [14]:
resolution.style.background_gradient(cmap ='YlOrRd',axis=None)

### Energy of the Cesium peak using the current calibration (March 2023, 888, 100ns)

In [15]:
fit = spectra.applymap(lambda x: abs(get_resolution(x)[2]-662)/662*100)

Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 


In [16]:
fit.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

### Count rate using integral <250keV (Calibration: March 2023, 888, 100ns)

In [17]:
cps = spectra.applymap(lambda x: x.Rate(300))

In [18]:
cps.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [19]:
cps.style.background_gradient(cmap ='YlOrRd',axis=None)

### Alpha activity rate using 1600keV < E < 2800keV (Calibration: March 2023, 888, 100ns)

In [20]:
alpha = spectra.applymap(lambda x: x.Alpha(300))

In [21]:
alpha.T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [22]:
alpha.style.background_gradient(cmap ='YlOrRd',axis=None)

## Correlations

In [23]:
df = pandas.concat([resolution,alpha,cps,fit],keys=["Resolution","Alpha","Rate","Fit"]).drop([100,150,200], level=3, axis=0).T

In [24]:
df.corr().style.background_gradient(cmap ='YlOrRd',axis=None)

## Draw fits

In [25]:
canvas = spectra.applymap(TH1D_draw)
#canvas.applymap(lambda x: type(x).__name__)

Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 


In [26]:
#for col in canvas.columns:
#    for row in canvas.index:
#        canvas.loc[row,col].Draw()

for CW in [100,150,200,250]:
    for run in [2]:
        for crystal in [3]:
            canvas.loc[(run,"888",CW),("B",crystal)].Draw()

To Do:
- Background subtraction
- Proper calibration
- 88c configuration and analysis